In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

# scikit learn의 LinearRegression 모델을 사용한 선형회귀 풀이
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

In [2]:
N = 1000 # 데이터의 수. M, N은 매우 커지는 상황을 가정할 것임
M = 3 # feature 수
rng = np.random.RandomState(1)
rng.rand(N, M).shape
X = 10 * rng.rand(N, M)
# [1.5, -2, 1] : Weight W를 임의로 이렇게 설정한 것.
# 해당 weight에 input data X를 행렬곱한 것이 정답 label이 된다
# 그리고 model은 계수를 학습하여 우리가 설정한 W에 가깝게 근사시킨 값을 리턴하면 된다
np.dot(X, [1.5, -2., 1.]).shape

(1000,)

In [4]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(N, M)
y = 0.5 + np.dot(X, [1.5, -2., 1.])

model.fit(X, y)
print(model.intercept_)
# 우리가 설정했던 W 계수와 일치하는 계수값을 가지도록 학습되었음을 알 수 있다
print(model.coef_)

0.499999999999968
[ 1.5 -2.   1. ]


### Normal Equations

In [5]:
# Normal equation 식을 직접 작성하여 선형회귀를 풀이하는 방식
import numpy.linalg as LA
normal_equation_solution = LA.inv(X.T@X)@X.T@y

In [6]:
# 위와는 bias 정도의 차이가 발생한듯?
normal_equation_solution

array([ 1.52825484, -1.96886193,  1.03058857])

### Small Memory Normal Equations (Online)

In [7]:
# 대규모의 데이터를 가정한 경우의 학습방식.
# 위에서는 한번에 X를 행렬 곱해버림. 그러나 대규모의 데이터로 메모리에 무리가 갈 수준의 큰 데이터라면
# 위처럼 한 번에 X.T @ X 이런 식의 연산이 불가능함.
# 따라서 위의 행렬식을 input data X행렬의 i번째 행(열)벡터 간 외적의 합으로 변형하여 점진적 online 학습이 진행되도록 만들어준다. 
A = np.zeros([M, M])
b = np.zeros([M, 1])

In [8]:
for i in range(N):
    A = A + X[i, np.newaxis].T@X[i, np.newaxis]
    b = b + X[i, np.newaxis].T*y[i]
solution = LA.inv(A)@b

In [9]:
solution

array([[ 1.52825484],
       [-1.96886193],
       [ 1.03058857]])

### SGD

In [10]:
# stochastic gradient descent
w = np.zeros([M, 1]) # W값을 0으로 초기화, M개의 dimension(feature)

In [11]:
eta = 0.001 # Learning Rate
n_iter = 1000 # 파라미터를 몇 번 업데이트 할 것인가 = 위에서 데이터의 수가 1000개이므로 1000번으로 설정했다.

In [12]:
for i in range(n_iter):
    i = i % N # i가 데이터의 수를 넘어가게 될 경우에 다시 처음부터
    neg_gradient = (y[i] - w.T@X[i, np.newaxis].T) * X[i, np.newaxis].T
    w = w + eta * neg_gradient
print(w)

[[ 1.5271529 ]
 [-1.97170249]
 [ 1.0336901 ]]
